# Library

In [1]:
# Native library
import copy
import warnings
warnings.filterwarnings('ignore')

# Save object
import joblib

# Data management
import numpy as np
import pandas as pd

# Data visualization
import plotly.express as px

# Data prepocessing
# from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Hyperoptimization
from sklearn.model_selection import GridSearchCV

# Regressor models
from xgboost import XGBRegressor

# Model evaluation
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

In [2]:
# Target variable
TARGET = "Rice Yield (kg/ha)"

# Import Data

In [3]:
df_vi = pd.read_csv("../../data/processed/fixed_0-00146/train_vi.csv")

df_vi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13368 entries, 0 to 13367
Data columns (total 17 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   District                                        13368 non-null  object 
 1   Latitude                                        13368 non-null  float64
 2   Longitude                                       13368 non-null  float64
 3   Season(SA = Summer Autumn, WS = Winter Spring)  13368 non-null  object 
 4   Rice Crop Intensity(D=Double, T=Triple)         13368 non-null  object 
 5   Date of Harvest                                 13368 non-null  object 
 6   Field size (ha)                                 13368 non-null  float64
 7   Rice Yield (kg/ha)                              13368 non-null  int64  
 8   date                                            13368 non-null  object 
 9   ndvi                                   

In [4]:
df_sat = pd.read_csv("../../data/processed/fixed_0-00146/train.csv")

df_sat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13368 entries, 0 to 13367
Data columns (total 17 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   District                                        13368 non-null  object 
 1   Latitude                                        13368 non-null  float64
 2   Longitude                                       13368 non-null  float64
 3   Season(SA = Summer Autumn, WS = Winter Spring)  13368 non-null  object 
 4   Rice Crop Intensity(D=Double, T=Triple)         13368 non-null  object 
 5   Date of Harvest                                 13368 non-null  object 
 6   Field size (ha)                                 13368 non-null  float64
 7   Rice Yield (kg/ha)                              13368 non-null  int64  
 8   date                                            13368 non-null  object 
 9   red                                    

In [5]:
df = pd.merge(df_vi, df_sat, on=['District', 'Latitude', 'Longitude',
       'Season(SA = Summer Autumn, WS = Winter Spring)',
       'Rice Crop Intensity(D=Double, T=Triple)', 'Date of Harvest',
       'Field size (ha)', 'Rice Yield (kg/ha)', 'date'])

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13368 entries, 0 to 13367
Data columns (total 25 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   District                                        13368 non-null  object 
 1   Latitude                                        13368 non-null  float64
 2   Longitude                                       13368 non-null  float64
 3   Season(SA = Summer Autumn, WS = Winter Spring)  13368 non-null  object 
 4   Rice Crop Intensity(D=Double, T=Triple)         13368 non-null  object 
 5   Date of Harvest                                 13368 non-null  object 
 6   Field size (ha)                                 13368 non-null  float64
 7   Rice Yield (kg/ha)                              13368 non-null  int64  
 8   date                                            13368 non-null  object 
 9   ndvi                                   

In [6]:
df.head()

,District,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha),Rice Yield (kg/ha),date,ndvi,...,mtvi1,lswi,red,green,blue,rededge1,rededge2,rededge3,nir,swir
0,Chau_Phu,10.510542,105.248554,SA,T,15-07-2022,3.4,5500,14-07-2022,0.182807,...,3062.295791,0.276005,3707.568604,4139.405273,4338.941406,4399.774414,5063.293945,5313.826660,5366.339844,3044.820312
1,Chau_Phu,10.510542,105.248554,SA,T,15-07-2022,3.4,5500,09-07-2022,0.453044,...,5106.577866,0.266163,1954.647095,2239.111084,1972.908447,2674.836670,4337.316895,5363.189453,5192.712402,3009.568604
2,Chau_Phu,10.510542,105.248554,SA,T,15-07-2022,3.4,5500,04-07-2022,0.125385,...,2728.251445,0.210415,5195.346191,5569.202637,5768.026367,5971.241699,6526.993652,6799.486816,6684.954102,4360.764648
3,Chau_Phu,10.510542,105.248554,SA,T,15-07-2022,3.4,5500,29-06-2022,0.176642,...,3024.090264,0.345635,3937.790771,4316.660156,4571.542480,4673.444336,5374.594727,5831.339844,5627.411621,2736.535889
4,Chau_Phu,10.510542,105.248554,SA,T,15-07-2022,3.4,5500,24-06-2022,0.452227,...,4482.325225,0.234559,1723.215698,1970.081665,1815.310425,2227.434570,3854.986816,4699.708984,4568.503418,2832.526123


## Renaming & Typing

In [7]:
# Rename columns
# df.rename(
#     {
#         "District": "district", 
#         "Latitude": "lat", 
#         "Longitude": "lon",
#         "Season(SA = Summer Autumn, WS = Winter Spring)": "season",
#         "Rice Crop Intensity(D=Double, T=Triple)": "intensity",
#         "Date of Harvest": "harvest",
#         "Field size (ha)": "size",
#         "Rice Yield (kg/ha)": "yield",
#         # "ID No": "id_no"
#     },
#     axis="columns", inplace=True)

# Set the type of each columns
df = df.astype(
    dtype={
        # "id_no": "category",
        "Rice Crop Intensity(D=Double, T=Triple)": "category",
    },
   copy=False 
)

#Remove unsuful columns
# df.drop(columns=["district", "lat", "lon", "season"], inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13368 entries, 0 to 13367
Data columns (total 25 columns):
 #   Column                                          Non-Null Count  Dtype   
---  ------                                          --------------  -----   
 0   District                                        13368 non-null  object  
 1   Latitude                                        13368 non-null  float64 
 2   Longitude                                       13368 non-null  float64 
 3   Season(SA = Summer Autumn, WS = Winter Spring)  13368 non-null  object  
 4   Rice Crop Intensity(D=Double, T=Triple)         13368 non-null  category
 5   Date of Harvest                                 13368 non-null  object  
 6   Field size (ha)                                 13368 non-null  float64 
 7   Rice Yield (kg/ha)                              13368 non-null  int64   
 8   date                                            13368 non-null  object  
 9   ndvi                        

# Feature Engineering

## Numerical data

In [8]:
df = df.drop(columns="date").groupby(['District', 'Latitude', 'Longitude',
       'Season(SA = Summer Autumn, WS = Winter Spring)',
       'Rice Crop Intensity(D=Double, T=Triple)', 'Date of Harvest',
       'Field size (ha)', 'Rice Yield (kg/ha)']).describe()
df.columns = df.columns.map('_'.join).str.strip('_')

In [9]:
df.reset_index(level=["Field size (ha)", "Rice Yield (kg/ha)", "Rice Crop Intensity(D=Double, T=Triple)"], inplace=True)

# Data Preparation

In [10]:
df.head()

Rice Crop Intensity(D=Double, T=Triple)  \
District Latitude  Longitude  Season(SA = Summer Autumn, WS = Winter Spring) Date of Harvest                                           
Chau_Phu 10.445962 105.123164 WS                                             07-04-2022                                            D   
         10.447840 105.141425 SA                                             14-07-2022                                            D   
                              WS                                             03-04-2022                                            D   
         10.455882 105.126628 WS                                             10-04-2022                                            D   
         10.458071 105.162504 WS                                             03-04-2022                                            D   

                                                                                              Field size (ha)  \
District Latitude  Longitude  Season(SA = Summer Autumn, WS = Winter Spring) Date of Harvest                    
Chau_Phu 10.445962 105.123164 WS                                             07-04-2022                   3.0   
         10.447840 105.141425 SA                                             14-07-2022                   2.0   
                              WS                                             03-04-2022                   2.0   
         10.455882 105.126628 WS                                             10-04-2022                   1.9   
         10.458071 105.162504 WS                                             03-04-2022                   2.0   

                                                                                              Rice Yield (kg/ha)  \
District Latitude  Longitude  Season(SA = Summer Autumn, WS = Winter Spring) Date of Harvest                       
Chau_Phu 10.445962 105.123164 WS                                             07-04-2022                     7200   
         10.447840 105.141425 SA                                             14-07-2022                     6600   
                              WS                                             03-04-2022                     6400   
         10.455882 105.126628 WS                                             10-04-2022                     6800   
         10.458071 105.162504 WS                                             03-04-2022                     6400   

                                                                                              ndvi_count  \
District Latitude  Longitude  Season(SA = Summer Autumn, WS = Winter Spring) Date of Harvest               
Chau_Phu 10.445962 105.123164 WS                                             07-04-2022             24.0   
         10.447840 105.141425 SA                                             14-07-2022             24.0   
                              WS                                             03-04-2022             24.0   
         10.455882 105.126628 WS                                             10-04-2022             24.0   
         10.458071 105.162504 WS                                             03-04-2022             24.0   

                                                                                              ndvi_mean  \
District Latitude  Longitude  Season(SA = Summer Autumn, WS = Winter Spring) Date of Harvest              
Chau_Phu 10.445962 105.123164 WS                                             07-04-2022        0.313879   
         10.447840 105.141425 SA                                             14-07-2022        0.227551   
                              WS                                             03-04-2022        0.304275   
         10.455882 105.126628 WS                                             10-04-2022        0.297431   
         10.458071 105.162504 WS                                             03-04-2022        0.321277   

                            

In [11]:
# Split Features & Target
X, y = df.drop(columns=TARGET), df[[TARGET]]

## Convert Categorical features

In [12]:
# Convert categorical variable into dummy/indicator variables
X = pd.get_dummies(X, drop_first=True)
X.head()

Field size (ha)  \
District Latitude  Longitude  Season(SA = Summer Autumn, WS = Winter Spring) Date of Harvest                    
Chau_Phu 10.445962 105.123164 WS                                             07-04-2022                   3.0   
         10.447840 105.141425 SA                                             14-07-2022                   2.0   
                              WS                                             03-04-2022                   2.0   
         10.455882 105.126628 WS                                             10-04-2022                   1.9   
         10.458071 105.162504 WS                                             03-04-2022                   2.0   

                                                                                              ndvi_count  \
District Latitude  Longitude  Season(SA = Summer Autumn, WS = Winter Spring) Date of Harvest               
Chau_Phu 10.445962 105.123164 WS                                             07-04-2022             24.0   
         10.447840 105.141425 SA                                             14-07-2022             24.0   
                              WS                                             03-04-2022             24.0   
         10.455882 105.126628 WS                                             10-04-2022             24.0   
         10.458071 105.162504 WS                                             03-04-2022             24.0   

                                                                                              ndvi_mean  \
District Latitude  Longitude  Season(SA = Summer Autumn, WS = Winter Spring) Date of Harvest              
Chau_Phu 10.445962 105.123164 WS                                             07-04-2022        0.313879   
         10.447840 105.141425 SA                                             14-07-2022        0.227551   
                              WS                                             03-04-2022        0.304275   
         10.455882 105.126628 WS                                             10-04-2022        0.297431   
         10.458071 105.162504 WS                                             03-04-2022        0.321277   

                                                                                              ndvi_std  \
District Latitude  Longitude  Season(SA = Summer Autumn, WS = Winter Spring) Date of Harvest             
Chau_Phu 10.445962 105.123164 WS                                             07-04-2022       0.278279   
         10.447840 105.141425 SA                                             14-07-2022       0.197603   
                              WS                                             03-04-2022       0.289463   
         10.455882 105.126628 WS                                             10-04-2022       0.283660   
         10.458071 105.162504 WS                                             03-04-2022       0.266027   

                                                                                              ndvi_min  \
District Latitude  Longitude  Season(SA = Summer Autumn, WS = Winter Spring) Date of Harvest             
Chau_Phu 10.445962 105.123164 WS                                             07-04-2022      -0.070971   
         10.447840 105.141425 SA                                             14-07-2022      -0.070756   
                              WS                                             03-04-2022      -0.070756   
         10.455882 105.126628 WS                                             10-04-2022      -0.079871   
         10.458071 105.162504 WS                                             03-04-2022      -0.077437   

                                                                                              ndvi_25%  \
District Latitude  Longitude  Season(SA = Summer Autumn, WS = Winter Spring) Date of Harvest             
Chau_Phu 10.445962 105.123164 WS                                             07-04-2022       0.0

## Split Data

In [13]:
# Split dataset into Train/Test subdataset equitably distributed according to TARGET
# Repeat the operation to create Train/Val/Test subdataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Scale Data

Because XGBoost is a decision tree based algorithm it not required normalization of their inputs.

In [14]:
# # Depend of the model used a data scaling is requered
# scaler = StandardScaler()
# X_train_scale = scaler.fit_transform(X_train)

# # X_train = pd.DataFrame(scaler.transform(X_train), columns=X.columns)
# # X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)
# X_test_scale = scaler.transform(X_test)

# # Save scaler for a future use
# scaler_filename = "scaler.save"
# joblib.dump(scaler, scaler_filename) 
# # to load it
# # scaler = joblib.load(scaler_filename) 

# X_train.head()

# Machine Learning Regression

## XGBoost

### Train

In [15]:
xgbr = XGBRegressor(random_state=0)

param_grid = { 
    'max_depth' : np.linspace(2, 10, 5, dtype=int),
    'n_estimators': np.linspace(100, 1000, 5, dtype=int),
    'learning_rate': np.linspace(0.01, 0.3, 5, dtype=float),
    'colsample_bytree': np.linspace(0.5, 1, 5, dtype=float),
    # 'subsample': np.linspace(0.6, 1, 5, dtype=float),
}

# Perform a grid search
cv_xgbr = GridSearchCV(xgbr, param_grid, n_jobs=-1, verbose=1)
cv_xgbr.fit(X_train, y_train)

Fitting 5 folds for each of 625 candidates, totalling 3125 fits


GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    gamma=None, gpu_id=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bin=None,
                                    max_cat_to_o...
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=0,
                                    reg_alpha=None, reg_lambda=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': array([0.5  , 0.625, 0.75 , 0.875, 1.   ]),
                         'learning_rate': array([0.01  , 0.0825, 0.155 , 0.2275, 0.3   ]),
                         'max_depth': array([ 2,  4,  6,  8, 10]),
                         'n_estimators': array([ 100,  325,  550,  775, 1000])},
             verbose=1)

In [16]:
cv_xgbr.best_params_

{'colsample_bytree': 0.625,
 'learning_rate': 0.01,
 'max_depth': 2,
 'n_estimators': 550}

### Evaluate

In [17]:
y_pred = cv_xgbr.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
mape = mean_absolute_percentage_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'RMSE: {rmse:.0f} | MAPE: {100*mape:.1f}% | R2 score: {r2:.3f}')

RMSE: 483 | MAPE: 6.0% | R2 score: 0.690


In [18]:
# Plot features importance
df_fi = pd.DataFrame(columns=['Feature', 'Importance'])
df_fi['Feature'] = X.columns
df_fi['Importance'] = cv_xgbr.best_estimator_.feature_importances_
df_fi.sort_values('Importance', inplace=True, ascending=False)

fig = px.bar(df_fi.head(10), x='Feature', y='Importance', title="Feature importance")
fig.show()

# Save the model

In [19]:
model_filename = "../model/fixed_0-00146/cv_xgboost.save"
joblib.dump(cv_xgbr, model_filename)

['../model/fixed_0-00146/cv_xgboost.save']

# Train Final Model

In [21]:
param = copy.deepcopy(cv_xgbr.best_params_)
param["random_state"] = 0

xgbr = XGBRegressor(**param)

xgbr.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.625,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.01, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=2, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=550, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [22]:
model_filename = "../model/fixed_0-00146/xgboost.save"
joblib.dump(xgbr, model_filename)

['../model/fixed_0-00146/xgboost.save']

# Test file

In [23]:
df_vi = pd.read_csv("../../data/processed/fixed_0-00146/test_vi.csv")

df_vi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 18 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   ID No                                           2400 non-null   int64  
 1   District                                        2400 non-null   object 
 2   Latitude                                        2400 non-null   float64
 3   Longitude                                       2400 non-null   float64
 4   Season(SA = Summer Autumn, WS = Winter Spring)  2400 non-null   object 
 5   Rice Crop Intensity(D=Double, T=Triple)         2400 non-null   object 
 6   Date of Harvest                                 2400 non-null   object 
 7   Field size (ha)                                 2400 non-null   float64
 8   Predicted Rice Yield (kg/ha)                    0 non-null      float64
 9   date                                     

In [24]:
df_sat = pd.read_csv("../../data/processed/fixed_0-00146/test.csv")

df_sat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 18 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   ID No                                           2400 non-null   int64  
 1   District                                        2400 non-null   object 
 2   Latitude                                        2400 non-null   float64
 3   Longitude                                       2400 non-null   float64
 4   Season(SA = Summer Autumn, WS = Winter Spring)  2400 non-null   object 
 5   Rice Crop Intensity(D=Double, T=Triple)         2400 non-null   object 
 6   Date of Harvest                                 2400 non-null   object 
 7   Field size (ha)                                 2400 non-null   float64
 8   Predicted Rice Yield (kg/ha)                    0 non-null      float64
 9   date                                     

In [25]:
df = pd.merge(df_vi, df_sat, on=['District', 'Latitude', 'Longitude',
       'Season(SA = Summer Autumn, WS = Winter Spring)',
       'Rice Crop Intensity(D=Double, T=Triple)', 'Date of Harvest',
       'Field size (ha)', 'Predicted Rice Yield (kg/ha)', 'date', 'ID No'])

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2400 entries, 0 to 2399
Data columns (total 26 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   ID No                                           2400 non-null   int64  
 1   District                                        2400 non-null   object 
 2   Latitude                                        2400 non-null   float64
 3   Longitude                                       2400 non-null   float64
 4   Season(SA = Summer Autumn, WS = Winter Spring)  2400 non-null   object 
 5   Rice Crop Intensity(D=Double, T=Triple)         2400 non-null   object 
 6   Date of Harvest                                 2400 non-null   object 
 7   Field size (ha)                                 2400 non-null   float64
 8   Predicted Rice Yield (kg/ha)                    0 non-null      float64
 9   date                                     

In [26]:
df.head()

,ID No,District,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha),Predicted Rice Yield (kg/ha),date,...,mtvi1,lswi,red,green,blue,rededge1,rededge2,rededge3,nir,swir
0,1,Chau_Phu,10.542192,105.18792,WS,T,10-04-2022,1.4,NaN,10-04-2022,...,648.663369,-0.060031,3381.496826,3313.457520,3360.209229,3762.447754,3857.467285,4083.084961,3905.666748,4404.532715
1,1,Chau_Phu,10.542192,105.18792,WS,T,10-04-2022,1.4,NaN,05-04-2022,...,2375.349390,0.063495,2023.441162,1984.607788,1656.869263,2676.281006,3438.506592,3771.326904,3715.058838,3271.450928
2,1,Chau_Phu,10.542192,105.18792,WS,T,10-04-2022,1.4,NaN,31-03-2022,...,3644.730498,0.148839,2205.650391,2361.464111,2057.800537,2986.781006,4082.418213,4653.062012,4567.915039,3384.313721
3,1,Chau_Phu,10.542192,105.18792,WS,T,10-04-2022,1.4,NaN,26-03-2022,...,3737.763252,0.168246,1706.290894,1841.614380,1480.372559,2472.673096,3676.718994,4131.218750,4155.359375,2958.483643
4,1,Chau_Phu,10.542192,105.18792,WS,T,10-04-2022,1.4,NaN,21-03-2022,...,3085.443384,0.198814,1734.813721,1941.666626,1757.101318,2280.630615,3256.418213,3770.830078,3653.392090,2441.620850


## Renaming & Typing

In [27]:
# Rename columns
# df.rename(
#     {
#         "District": "district", 
#         "Latitude": "lat", 
#         "Longitude": "lon",
#         "Season(SA = Summer Autumn, WS = Winter Spring)": "season",
#         "Rice Crop Intensity(D=Double, T=Triple)": "intensity",
#         "Date of Harvest": "harvest",
#         "Field size (ha)": "size",
#         "Rice Yield (kg/ha)": "yield",
#         "ID No": "id_no"
#     },
#     axis="columns", inplace=True)

# Set the type of each columns
df = df.astype(
    dtype={
        # "id_no": "category",
        "Rice Crop Intensity(D=Double, T=Triple)": "category",
    },
   copy=False 
)

#Remove unusuful columns
# df.drop(columns=["id_no"], inplace=True)

df.head()

,ID No,District,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha),Predicted Rice Yield (kg/ha),date,...,mtvi1,lswi,red,green,blue,rededge1,rededge2,rededge3,nir,swir
0,1,Chau_Phu,10.542192,105.18792,WS,T,10-04-2022,1.4,NaN,10-04-2022,...,648.663369,-0.060031,3381.496826,3313.457520,3360.209229,3762.447754,3857.467285,4083.084961,3905.666748,4404.532715
1,1,Chau_Phu,10.542192,105.18792,WS,T,10-04-2022,1.4,NaN,05-04-2022,...,2375.349390,0.063495,2023.441162,1984.607788,1656.869263,2676.281006,3438.506592,3771.326904,3715.058838,3271.450928
2,1,Chau_Phu,10.542192,105.18792,WS,T,10-04-2022,1.4,NaN,31-03-2022,...,3644.730498,0.148839,2205.650391,2361.464111,2057.800537,2986.781006,4082.418213,4653.062012,4567.915039,3384.313721
3,1,Chau_Phu,10.542192,105.18792,WS,T,10-04-2022,1.4,NaN,26-03-2022,...,3737.763252,0.168246,1706.290894,1841.614380,1480.372559,2472.673096,3676.718994,4131.218750,4155.359375,2958.483643
4,1,Chau_Phu,10.542192,105.18792,WS,T,10-04-2022,1.4,NaN,21-03-2022,...,3085.443384,0.198814,1734.813721,1941.666626,1757.101318,2280.630615,3256.418213,3770.830078,3653.392090,2441.620850


# Feature Engineering

## Numerical data

In [28]:
df = df.drop(columns=["date", 'Predicted Rice Yield (kg/ha)']).groupby(['District', 'Latitude', 'Longitude',
       'Season(SA = Summer Autumn, WS = Winter Spring)',
       'Rice Crop Intensity(D=Double, T=Triple)', 'Date of Harvest',
       'Field size (ha)', 'ID No']).describe()

df.columns = df.columns.map('_'.join).str.strip('_')

In [29]:
df.reset_index(level=["Field size (ha)", "Rice Crop Intensity(D=Double, T=Triple)"], inplace=True)

In [30]:
df.head()

Rice Crop Intensity(D=Double, T=Triple)  \
District Latitude  Longitude  Season(SA = Summer Autumn, WS = Winter Spring) Date of Harvest ID No                                           
Chau_Phu 10.441423 105.115088 SA                                             20-07-2022      17                                          D   
         10.469839 105.211568 WS                                             01-04-2022      85                                          T   
         10.473786 105.190479 SA                                             14-07-2022      13                                          T   
                              WS                                             03-04-2022      35                                          T   
         10.474439 105.216928 SA                                             15-07-2022      81                                          T   

                                                                                                    Field size (ha)  \
District Latitude  Longitude  Season(SA = Summer Autumn, WS = Winter Spring) Date of Harvest ID No                    
Chau_Phu 10.441423 105.115088 SA                                             20-07-2022      17                 4.0   
         10.469839 105.211568 WS                                             01-04-2022      85                 4.0   
         10.473786 105.190479 SA                                             14-07-2022      13                 1.7   
                              WS                                             03-04-2022      35                 1.7   
         10.474439 105.216928 SA                                             15-07-2022      81                 7.0   

                                                                                                    ndvi_count  \
District Latitude  Longitude  Season(SA = Summer Autumn, WS = Winter Spring) Date of Harvest ID No               
Chau_Phu 10.441423 105.115088 SA                                             20-07-2022      17           24.0   
         10.469839 105.211568 WS                                             01-04-2022      85           24.0   
         10.473786 105.190479 SA                                             14-07-2022      13           24.0   
                              WS                                             03-04-2022      35           24.0   
         10.474439 105.216928 SA                                             15-07-2022      81           24.0   

                                                                                                    ndvi_mean  \
District Latitude  Longitude  Season(SA = Summer Autumn, WS = Winter Spring) Date of Harvest ID No              
Chau_Phu 10.441423 105.115088 SA                                             20-07-2022      17      0.245517   
         10.469839 105.211568 WS                                             01-04-2022      85      0.298268   
         10.473786 105.190479 SA                                             14-07-2022      13      0.250739   
                              WS                                             03-04-2022      35      0.334348   
         10.474439 105.216928 SA                                             15-07-2022      81      0.233725   

                                                                                                    ndvi_std  \
District Latitude  Longitude  Season(SA = Summer Autumn, WS = Winter Spring) Date of Harvest ID No             
Chau_Phu 10.441423 105.115088 SA                                             20-07-2022      17     0.230892   
         10.469839 105.211568 WS                                             01-04-2022      85     0.246330   
         10.473786 105.190479 SA                                             14-07-2022      13     0.231802   
                              WS                                             03-04-2022      35     0.268628   
       

# Data Preparation

In [31]:
# Split Features & Target
X = df

## Convert Categorical features

In [32]:
# Convert categorical variable into dummy/indicator variables
X = pd.get_dummies(X, drop_first=True)
X.head()

Field size (ha)  \
District Latitude  Longitude  Season(SA = Summer Autumn, WS = Winter Spring) Date of Harvest ID No                    
Chau_Phu 10.441423 105.115088 SA                                             20-07-2022      17                 4.0   
         10.469839 105.211568 WS                                             01-04-2022      85                 4.0   
         10.473786 105.190479 SA                                             14-07-2022      13                 1.7   
                              WS                                             03-04-2022      35                 1.7   
         10.474439 105.216928 SA                                             15-07-2022      81                 7.0   

                                                                                                    ndvi_count  \
District Latitude  Longitude  Season(SA = Summer Autumn, WS = Winter Spring) Date of Harvest ID No               
Chau_Phu 10.441423 105.115088 SA                                             20-07-2022      17           24.0   
         10.469839 105.211568 WS                                             01-04-2022      85           24.0   
         10.473786 105.190479 SA                                             14-07-2022      13           24.0   
                              WS                                             03-04-2022      35           24.0   
         10.474439 105.216928 SA                                             15-07-2022      81           24.0   

                                                                                                    ndvi_mean  \
District Latitude  Longitude  Season(SA = Summer Autumn, WS = Winter Spring) Date of Harvest ID No              
Chau_Phu 10.441423 105.115088 SA                                             20-07-2022      17      0.245517   
         10.469839 105.211568 WS                                             01-04-2022      85      0.298268   
         10.473786 105.190479 SA                                             14-07-2022      13      0.250739   
                              WS                                             03-04-2022      35      0.334348   
         10.474439 105.216928 SA                                             15-07-2022      81      0.233725   

                                                                                                    ndvi_std  \
District Latitude  Longitude  Season(SA = Summer Autumn, WS = Winter Spring) Date of Harvest ID No             
Chau_Phu 10.441423 105.115088 SA                                             20-07-2022      17     0.230892   
         10.469839 105.211568 WS                                             01-04-2022      85     0.246330   
         10.473786 105.190479 SA                                             14-07-2022      13     0.231802   
                              WS                                             03-04-2022      35     0.268628   
         10.474439 105.216928 SA                                             15-07-2022      81     0.203865   

                                                                                                    ndvi_min  \
District Latitude  Longitude  Season(SA = Summer Autumn, WS = Winter Spring) Date of Harvest ID No             
Chau_Phu 10.441423 105.115088 SA                                             20-07-2022      17    -0.084224   
         10.469839 105.211568 WS                                             01-04-2022      85    -0.064239   
         10.473786 105.190479 SA                                             14-07-2022      13    -0.115478   
                              WS                                             03-04-2022      35    -0.064958   
         10.474439 105.216928 SA                                             15-07-2022      81    -0.070384   

                                                                                                    ndvi_

# Load the model

In [33]:
model_filename = "../model/fixed_0-00146/xgboost.save"
model = joblib.load(model_filename)

In [34]:
y_pred = model.predict(X)

In [35]:
s_pred = pd.Series(y_pred, index=df.index, name='Predicted Rice Yield (kg/ha)')

# Load Submissions file

In [36]:
df_sub = pd.read_csv("../../data/raw/test.csv")
df_sub.drop(columns='Predicted Rice Yield (kg/ha)', inplace=True)
df_sub.head()

,ID No,District,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha)
0,1,Chau_Phu,10.542192,105.187920,WS,T,10-04-2022,1.40
1,2,Chau_Thanh,10.400189,105.331053,SA,T,15-07-2022,1.32
2,3,Chau_Phu,10.505489,105.203926,SA,D,14-07-2022,1.40
3,4,Chau_Phu,10.523520,105.138274,WS,D,10-04-2022,1.80
4,5,Thoai_Son,10.294660,105.248528,SA,T,20-07-2022,2.20


In [37]:
df_sub = pd.merge(df_sub, s_pred, on="ID No")
df_sub.head()

,ID No,District,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha),Predicted Rice Yield (kg/ha)
0,1,Chau_Phu,10.542192,105.187920,WS,T,10-04-2022,1.40,7184.517578
1,2,Chau_Thanh,10.400189,105.331053,SA,T,15-07-2022,1.32,5890.771973
2,3,Chau_Phu,10.505489,105.203926,SA,D,14-07-2022,1.40,5966.685547
3,4,Chau_Phu,10.523520,105.138274,WS,D,10-04-2022,1.80,7173.996094
4,5,Thoai_Son,10.294660,105.248528,SA,T,20-07-2022,2.20,5812.468262


In [38]:
sub_filename = '../model/fixed_0-00146/submission.csv'
df_sub.to_csv(sub_filename, index=False, header=True)